# Zero Coupon Bond Curve

## Imports

In [2]:
import numpy as np
import pandas as pd
import plotly.io as pio

from carry_trade import utils

pd.options.plotting.backend = "plotly"
pio.templates.default = "none"

## Summary

In [3]:
start_date = "2014-12-01"
tickers = ["THA", "ROU", "JPN", "IDN"]
libors = ["JPY3MTD156N"]
currencies = ["THB", "RON", "JPY", "IDR"]

## Thai Baht YC/THA

In [4]:
dfs_yc = {t: pd.read_csv(f"data/df_{t}.csv") for t in tickers}
dfs_fx = {c: pd.read_csv(f"data/df_fx_{c}.csv") for c in currencies}
dfs_libor = {l: pd.read_csv(f"data/df_libor_{l}.csv") for l in libors}
for d in [dfs_yc, dfs_fx, dfs_libor]:
    for df in d.values():
        df["date"] = pd.to_datetime(df.date)
        df.set_index("date", inplace=True)

dfs_libor["JPY3MTD156N"].value = dfs_libor["JPY3MTD156N"].value.replace(".", None).astype(float)

In [5]:
week_start = pd.Timestamp("2021-04-21")
week_end = week_start + pd.DateOffset(days=7)
notional_start_USD = float(10e6)
capital_start_USD = float(2e6)
borrow_start_USD = notional_start_USD - capital_start_USD

Borrow $8MM notional JPY

In [6]:
borrow_rate = dfs_libor["JPY3MTD156N"].loc[week_start].value
fx_rate_start_borrow_USD = dfs_fx["JPY"].loc[week_start].rate
borrow_start_borrow = borrow_start_USD * fx_rate_start_borrow_USD

Exchange $2MM USD for JPY

In [7]:
capital_start_borrow = capital_start_USD * fx_rate_start_borrow_USD

Exchange $10MM USD notional JPY for Thai Baht

In [8]:
fx_rate_start_lend_USD = dfs_fx["THB"].loc[week_start].rate
fx_rate_start_lend_borrow = fx_rate_start_lend_USD / fx_rate_start_borrow_USD
notional_start_lend = (borrow_start_borrow + capital_start_borrow) * fx_rate_start_lend_borrow
notional_start_lend

313199980.0

Buy $10MM USD notional Thai Government Bonds

In [9]:
yield_start_lend = dfs_yc["THA"].loc[week_start, "5-year"]
yield_start_lend

0.984

In [10]:
s_cash_flow = utils.get_bond_cash_flows(yield_start_lend, notional_start_lend, week_start)
s_cash_flow

coupon_date
2021-07-21    7.704720e+07
2021-10-21    7.704720e+07
2022-01-21    7.704720e+07
2022-04-21    7.704720e+07
2022-07-21    7.704720e+07
2022-10-21    7.704720e+07
2023-01-21    7.704720e+07
2023-04-21    7.704720e+07
2023-07-21    7.704720e+07
2023-10-21    7.704720e+07
2024-01-21    7.704720e+07
2024-04-21    7.704720e+07
2024-07-21    7.704720e+07
2024-10-21    7.704720e+07
2025-01-21    7.704720e+07
2025-04-21    7.704720e+07
2025-07-21    7.704720e+07
2025-10-21    7.704720e+07
2026-01-21    7.704720e+07
2026-04-21    3.902472e+08
Name: cash_flow, dtype: float64

Sell Thai Government Bonds

In [11]:
week_end

Timestamp('2021-04-28 00:00:00')

In [12]:
yc_end = dfs_yc["THA"].loc[week_end].iloc[:8].astype(float)

In [16]:
utils.get_coupon_dates(week_end)

0    2021-07-28
1    2021-10-28
2    2022-01-28
3    2022-04-28
4    2022-07-28
5    2022-10-28
6    2023-01-28
7    2023-04-28
8    2023-07-28
9    2023-10-28
10   2024-01-28
11   2024-04-28
12   2024-07-28
13   2024-10-28
14   2025-01-28
15   2025-04-28
16   2025-07-28
17   2025-10-28
18   2026-01-28
19   2026-04-28
Name: coupon_date, dtype: datetime64[ns]

In [17]:
yc_dates = []
for i in dfs_yc["THA"].loc[week_end].iloc[:8].index:
    n, per = i.split("-")
    yc_dates.append(week_end + pd.DateOffset(**{f"{per}s": int(n)}))
yc_end.index = yc_dates
yc_end.name = "spot_rate"

In [18]:
print(dfs_yc["THA"].loc[week_end].iloc[:8].astype(float).to_frame().to_html())

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>2021-04-28 00:00:00</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1-month</th>
      <td>0.3020</td>
    </tr>
    <tr>
      <th>3-month</th>
      <td>0.3287</td>
    </tr>
    <tr>
      <th>6-month</th>
      <td>0.4145</td>
    </tr>
    <tr>
      <th>1-year</th>
      <td>0.4500</td>
    </tr>
    <tr>
      <th>2-year</th>
      <td>0.4760</td>
    </tr>
    <tr>
      <th>3-year</th>
      <td>0.7080</td>
    </tr>
    <tr>
      <th>4-year</th>
      <td>0.9150</td>
    </tr>
    <tr>
      <th>5-year</th>
      <td>1.0010</td>
    </tr>
  </tbody>
</table>


In [19]:
dfs_yc["THA"]

,1-month,3-month,6-month,1-year,2-year,3-year,4-year,5-year,6-year,7-year,8-year,9-year,10-year,15-year,ticker
date,,,,,,,,,,,,,,,
2014-12-01,2.0130,2.0200,2.0200,2.010,2.075,2.095,2.232,2.310,2.383,3.301,2.696,2.738,3.304,3.231,THA
2014-12-02,2.0110,2.0200,2.0200,2.000,2.079,2.078,2.235,2.304,2.374,3.301,2.693,2.738,3.304,3.198,THA
2014-12-03,2.0090,2.0100,2.0100,2.000,2.076,2.085,2.219,2.328,2.386,3.301,2.687,2.753,2.919,3.237,THA
2014-12-04,2.0090,2.0100,2.0100,2.000,2.074,2.091,2.243,2.329,2.403,3.301,2.702,2.752,2.911,3.227,THA
2014-12-05,2.0090,2.0100,2.0100,2.000,2.074,2.091,2.243,2.329,2.403,3.301,2.702,2.752,2.911,3.227,THA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-23,0.3012,0.3252,0.4050,0.446,0.413,0.686,0.905,0.993,NaN,NaN,1.548,NaN,1.850,2.127,THA
2021-04-26,0.3003,0.3253,0.4089,0.446,0.468,0.676,0.888,0.970,NaN,NaN,1.517,NaN,1.792,2.091,THA
2021-04-27,0.3002,0.3268,0.4113,0.449,0.476,0.694,0.905,0.996,NaN,NaN,1.553,NaN,1.828,2.121,THA


In [20]:
yc_end = yc_end.to_frame().reindex(utils.get_coupon_dates(week_end)).interpolate()

In [21]:
print((yc_end.iloc[:5]).to_html())

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>spot_rate</th>
    </tr>
    <tr>
      <th>coupon_date</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2021-07-28</th>
      <td>0.32870</td>
    </tr>
    <tr>
      <th>2021-10-28</th>
      <td>0.41450</td>
    </tr>
    <tr>
      <th>2022-01-28</th>
      <td>0.43225</td>
    </tr>
    <tr>
      <th>2022-04-28</th>
      <td>0.45000</td>
    </tr>
    <tr>
      <th>2022-07-28</th>
      <td>0.45650</td>
    </tr>
  </tbody>
</table>


In [22]:
t_rates = yc_end.iloc[:5]
t_times = (t_rates.index - week_end).days / 364
t_rates, t_times

(             spot_rate
 coupon_date           
 2021-07-28     0.32870
 2021-10-28     0.41450
 2022-01-28     0.43225
 2022-04-28     0.45000
 2022-07-28     0.45650,
 Float64Index([              0.25, 0.5027472527472527, 0.7554945054945055,
               1.0027472527472527, 1.2527472527472527],
              dtype='float64', name='coupon_date'))

This is an attempt to describe in detail the mechanics of calculating a zero coupon bond curve from a spot rates curve.

* The rates we get from YC are annual spot rates for bonds at various tenors with par values of 100 as of the date of the curve.
* Per the homework instructions, we assume coupons are paid quarterly.
* The task at hand is to calculate the discount rate for each coupon, using the earliest rates already on a zero coupon basis (1 year or less) to start with, such that we end up with the same present value at each tenor.
* You do that through a process called [bootstrapping or forward filling](https://en.wikipedia.org/wiki/Bootstrapping_(finance)), which essentially treats each coupon as if it were its own zero coupon bond and solves iteratively for the rate for each successive tenor.
* Below are illustrative calculations for the first calculated tenor (1.25 years) for the rates from YC\THA on 2021-04-28, which would then be repeated for each succssive tenor.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>2021-04-28</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1-month</th>
      <td>0.3020</td>
    </tr>
    <tr>
      <th>3-month</th>
      <td>0.3287</td>
    </tr>
    <tr>
      <th>6-month</th>
      <td>0.4145</td>
    </tr>
    <tr>
      <th>1-year</th>
      <td>0.4500</td>
    </tr>
    <tr>
      <th>2-year</th>
      <td>0.4760</td>
    </tr>
    <tr>
      <th>3-year</th>
      <td>0.7080</td>
    </tr>
    <tr>
      <th>4-year</th>
      <td>0.9150</td>
    </tr>
    <tr>
      <th>5-year</th>
      <td>1.0010</td>
    </tr>
  </tbody>
</table>

* Interpolate for the tenors where there aren't rates.


<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>spot_rate</th>
    </tr>
    <tr>
      <th>coupon_date</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2021-07-28</th>
      <td>0.32870</td>
    </tr>
    <tr>
      <th>2021-10-28</th>
      <td>0.41450</td>
    </tr>
    <tr>
      <th>2022-01-28</th>
      <td>0.43225</td>
    </tr>
    <tr>
      <th>2022-04-28</th>
      <td>0.45000</td>
    </tr>
    <tr>
      <th>2022-07-28</th>
      <td>0.45650</td>
    </tr>
    <tr>
      <th>2022-10-28</th>
      <td>0.46300</td>
    </tr>
    <tr>
      <th>2023-01-28</th>
      <td>0.46950</td>
    </tr>
    <tr>
      <th>2023-04-28</th>
      <td>0.47600</td>
    </tr>
  </tbody>
</table>



$$
\begin{align*}
0.114 \cdot e^{-0.3278\cdot{0.25}} + 0.114 \cdot e^{-0.4145\cdot{0.50}} + 0.114 \cdot e^{-0.4322\cdot{0.75}} + 0.114 \cdot e^{-0.4500\cdot{1}} + 1.114 \cdot e^{-r\cdot{1.25}} &= 1\\
\left( 0.114 \cdot e^{-0.3278\cdot{0.25}} + 0.114 \cdot e^{-0.4145\cdot{0.50}} + 0.114 \cdot e^{-0.4322\cdot{0.75}} + 0.114 \cdot e^{-0.4500\cdot{1}} \right) - 1 &= -1.114 \cdot e^{-r\cdot{1.25}} \\
\frac{1 - \left( 0.114 \cdot e^{-0.3278\cdot{0.25}} + 0.114 \cdot e^{-0.4145\cdot{0.50}} + 0.114 \cdot e^{-0.4322\cdot{0.75}} + 0.114 \cdot e^{-0.4500\cdot{1}} \right)}{1.114} &=  e^{-r\cdot{1.25}} \\
-\log \left( \frac{1 - \left( 0.114 \cdot e^{-0.3278\cdot{0.25}} + 0.114 \cdot e^{-0.4145\cdot{0.50}} + 0.114 \cdot e^{-0.4322\cdot{0.75}} + 0.114 \cdot e^{-0.4500\cdot{1}} \right)}{1.114} \right) \cdot \frac{1}{1.25} &= r \\
0.4334 = r

\end{align*}
$$

* Which is as follows in `compute_zcb_curve` in Zero_And_Spot_Curves.ipynb:
```python
-np.log((1 - preceding_coupons_val) / (1 + coupon_half_yr)) / tenor
```

In [25]:
- np.log((1 - 0.114125 * np.exp(-t_rates.spot_rate[:-1] * t_times[:-1]).sum())/ 1.114125) / 1.25

0.43452053049434464